In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import argparse
import imutils
import cv2
from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, LSTM, merge
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
import csv
from PIL import Image    
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tqdm import tqdm
import numpy as np # linear algebra
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
import collections
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout
## real time data augmentation ..

Generation_train = ImageDataGenerator(
		rotation_range = 5,          ## rotated image training
		height_shift_range = 0.1,   ## shifted image training
		rescale = 1.0/255,
		shear_range = 0.2,
		width_shift_range = 0.1,
		fill_mode = 'nearest')

Generation_test = ImageDataGenerator(
													rotation_range=0.05
                          rescale=1./255,
													fill_mode="nearest")
													     ### this is basically normalisation (ratio by which each pixel is multiplied)
trainGenerator = Generation_train.flow_from_directory(
			"D:\DevanagariHandwrittenCharacterDataset\Train",    ## directory of foler having classes of training sets
			color_mode = "grayscale",
			target_size = (32,32),
			batch_size = 32,
			class_mode = "categorical")
prev = ""
labels = ["ka","kha","ga","gha","kna","cha","chha","ja","jha","yna","t`a","t`ha","d`a","d`ha","adna","ta","tha","da","dha","na","pa","pha","ba","bha","ma","yaw","ra","la","waw","sha","shat","sa","ha","aksha","tra","gya","0","1","2","3","4","5","6","7","8","9"]
count = 0;

validation_generator = Generation_test.flow_from_directory(
			"D:\DevanagariHandwrittenCharacterDataset\Test",     ## directory of folder having tests set
			target_size=(32,32),
			batch_size=32,
			color_mode = "grayscale",
			class_mode= 'categorical')
			
model = Sequential()

##  introducing different layers + batch normalisation (gaussian curve)
#Layer1----------------------------------------------------------
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = (32,32,1), strides = 1, padding = "same"))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3,3), strides = 1, padding = "same"))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(512, (3,3), strides = 1,padding = "same"))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(512, (3,3), strides = 1,padding = "same"))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation = "relu", kernel_initializer = "uniform"))  # check for kernal initilaiser
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.5))
model.add(Dense(512 ,activation = "relu", kernel_initializer = "uniform"))  #
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.5))
model.add(Dense(46, activation = "softmax", kernel_initializer = "uniform"))  #  check whether it needs ac			
			
model.compile(optimizer = "adam",
		loss = "categorical_crossentropy",
		metrics = ["accuracy"])
		
print(model.summary())

res=model.fit_generator(
		Generation_train,
		epochs = 200,       
		validation_data = validation_generator,
		validation_steps = 432,
		steps_per_epoch = 200,
		)

%matplotlib inline
accu=res.history['accuracy']
#val_acc=res.history['val_accuracy']
loss=res.history['loss']
#val_loss=res.history['val_loss']

epochs=range(len(accu)) #No. of epochs

# graph plotting
import matplotlib.pyplot as plt
plt.plot(epochs,accu,'r',label='Training Accuracy')
plt.plot(epochs,val_acc,'g',label='Testing Accuracy')  
plt.legend()
plt.figure()

#Plot training and validation loss per epoch
plt.plot(epochs,loss,'r',label='Training Loss')
plt.plot(epochs,val_loss,'g',label='Testing Loss')   
plt.legend()
plt.show()
		
## saving the model --  we can do one thing is define two functions in a class..
# one function will have model preparation..
# the other will give results..

# model.save("HindiModel2.h5")

''' ["ka","kha","ga","gha","kna","cha","chha","ja","jha","yna","t`a","t`ha","d`a","d`ha","adna","ta","tha","da","dha","na","pa","pha","ba","bha","ma","yaw","ra","la","waw","sha","shat","sa","ha","aksha","tra","gya","0","1","2","3","4","5","6","7","8","9"]
labels =['yna', 't`aa', 't`haa', 'd`aa', 'd`haa', 'a`dna', 'ta', 'tha', 'da', 'dha', 'ka', 'na', 'pa', 'pha', 'ba', 'bha', 'ma', 'yaw', 'ra', 'la', 'waw', 'kha', 'sha', 'shat', 'sa', 'ha', 'aksha', 'tra', 'gya', 'ga', 'gha', 'kna', 'cha', 'chha', 'ja', 'jha', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
'''
labels = [u'\u091E',u'\u091F',u'\u0920',u'\u0921',u'\u0922',u'\u0923',u'\u0924',u'\u0925',u'\u0926',u'\u0927',u'\u0915',u'\u0928',u'\u092A',u'\u092B',u'\u092c',u'\u092d',u'\u092e',u'\u092f',u'\u0930',u'\u0932',u'\u0935',u'\u0916',u'\u0936',u'\u0937',u'\u0938',u'\u0939','ksha','tra','gya',u'\u0917',u'\u0918',u'\u0919',u'\u091a',u'\u091b',u'\u091c',u'\u091d',u'\u0966',u'\u0967',u'\u0968',u'\u0969',u'\u096a',u'\u096b',u'\u096c',u'\u096d',u'\u096e',u'\u096f']
#

# loading the 'captcha.jpg' image..
test_image = cv2.imread("205.png")   # to enter the name of image...
image = cv2.resize(test_image, (32,32))  # our kaggle dataset size is (32,32)
image = image.astype("float") / 255.0
image = img_to_array(image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#ret, image1 = cv2.threshold(image, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
image = np.expand_dims(image, axis=0)
image = np.expand_dims(image, axis=3)
model = tf.keras.models.load_model("HindiModel2.h5")  # directory of our model in my PC.
results = model.predict(image)[0]  ## only one letter per image...
print(labels[np.argmax(results)])
	